In [1]:
import pandas as pd
import os

### Dataset 
The dataset was released by NY Times, which contains a series of data files with cumulative counts of coronavirus cases in the United States at the state level. resource: https://github.com/nytimes/covid-19-data/blob/master/us-states.csv

In [2]:
#print out first 5 rows of data to get idea of content
df = pd.read_csv(os.getcwd()+ '/rawData/us-states.csv')
df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [3]:
#use of openclean's stream operator to avoid having to load the dataset into main-memory

from openclean.pipeline import stream

ds = stream(df)

### Data Profiling

Computing basic data profiling statistics help to get a better understanding for the dataset.

In [4]:
# Profile using the default data profiler.

from openclean.profiling.column import DefaultColumnProfiler
profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [5]:
#overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
date,21574,0,434,0.020117,8.664171
state,21574,0,55,0.002549,5.780448
fips,21574,0,55,0.002549,5.780448
cases,21574,0,18241,0.845508,13.875458
deaths,21574,0,8043,0.372810,11.700285


In [6]:
#the most frequent data type from the profiling results.

print('Schema:\n------')
for col in ds.columns:
    c = profiles.column(col)
    print("  '{}' ({})".format(col, c['datatypes']['distinct'].most_common(3)[0][0]))
    
# Print number of rows.
    
print('\n{:,} rows.'.format(ds.count()))


Schema:
------
  'date' (date)
  'state' (str)
  'fips' (int)
  'cases' (int)
  'deaths' (int)

21,574 rows.


### State Conversion¶

Converting name of States to Postal Abbreviation. Since Postal Abbreviation are capitalized letters and unique, it helps join with other dataset in future operations.

In [7]:
# Dictionary to transform States, Districts & Territories to Two-Letter codes and vice versa.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


In [8]:
df['state']= df['state'].map(us_state_abbrev)

In [9]:
df['state'].unique()

array(['WA', 'IL', 'CA', 'AZ', 'MA', 'WI', 'TX', 'NE', 'UT', 'OR', 'FL',
       'NY', 'RI', 'GA', 'NH', 'NC', 'NJ', 'CO', 'MD', 'NV', 'TN', 'HI',
       'IN', 'KY', 'MN', 'OK', 'PA', 'SC', 'DC', 'KS', 'MO', 'VT', 'VA',
       'CT', 'IA', 'LA', 'OH', 'MI', 'SD', 'AR', 'DE', 'MS', 'NM', 'ND',
       'WY', 'AK', 'ME', 'AL', 'ID', 'MT', 'PR', 'VI', 'GU', 'WV', 'MP'],
      dtype=object)

In [10]:
df

,date,state,fips,cases,deaths
0,2020-01-21,WA,53,1,0
1,2020-01-22,WA,53,1,0
2,2020-01-23,WA,53,1,0
3,2020-01-24,IL,17,1,0
4,2020-01-24,WA,53,1,0
...,...,...,...,...,...
21569,2021-03-29,VA,51,616509,10219
21570,2021-03-29,WA,53,365029,5296
21571,2021-03-29,WV,54,140991,2638
21572,2021-03-29,WI,55,634662,7278


### Data export

Exporting DataFrame to a csv file for further analysis.

In [11]:
df.to_csv(os.getcwd()+ '/cleanData/covidCase.csv',index=False)